In [96]:
import pandas as pd
import matplotlib.pyplot as plt
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time

In [254]:
## Connect to spotify
CLIENT_ID = 'f4b0975d0a7e419fb4ef187ddd061914'
CLIENT_SECRET = '54314e98d08d4581be9ed226e49dc251'
REDIRECT_URI = 'http://localhost/'
USERNAME = 'sgtpepurr'
SCOPES = [
    'user-modify-playback-state',
    'user-read-playback-state',
    'user-read-currently-playing',
    'user-follow-modify',
    'user-follow-read',
    'user-read-recently-played',
    'user-read-playback-position',
    'user-top-read',
    'playlist-read-collaborative',
    'playlist-modify-public',
    'playlist-read-private',
    'playlist-modify-private',
    'app-remote-control',
    'streaming',
    'user-library-modify',
    'user-library-read'
]
token = spotipy.util.prompt_for_user_token('sgtpepurr',SCOPES,client_id=CLIENT_ID,client_secret=CLIENT_SECRET,redirect_uri=REDIRECT_URI)
sp = spotipy.Spotify(auth=token)

In [69]:
# read rym data
rym_df = pd.read_csv('../data/rym-1950-2021.csv', sep='~')
rym_df.tail(10)

,Unnamed: 0.1,Unnamed: 0,Rank,Artist,Album,Date,Genres,RYM Rating,Ratings,Reviews
2470,950,30,31,Nas,Magic,24 December 2021,"East Coast Hip Hop, Boom Bap",3.67,3041.0,39.0
2471,951,31,32,Effluence,Psychocephalic Spawning,13 December 2021,"Avant-Garde Metal, Brutal Death Metal, Technic...",3.63,480.0,11.0
2472,952,32,33,"岡部啓一 [Keiichi Okabe], 帆足圭吾 [Keigo Hoashi], 石濱翔...",NieR Replicant ver.1.22474487139...,21 April 2021,"Video Game Music, Cinematic Classical, Neoclas...",3.61,407.0,5.0
2473,953,33,34,Wild Up,Julius Eastman Vol. 1: Femenine,18 June 2021,"Minimalism, Chamber Music",3.67,1290.0,9.0
2474,954,34,35,Ad Nauseam,Imperative Imperceptible Impulse,12 February 2021,"Technical Death Metal, Avant-Garde Metal",3.63,2620.0,30.0
2475,955,35,36,DJ Sabrina the Teenage DJ,The Makin' Magick II Album,26 November 2021,Outsider House,3.65,1099.0,4.0
2476,956,36,37,STOMACH BOOK,STOMACH BOOK,8 October 2021,"Experimental Rock, Noise Pop, Emo",3.66,606.0,2.0
2477,957,37,38,Skee Mask,Pool,7 May 2021,"IDM, Breakbeat, Techno",3.64,2604.0,13.0
2478,958,38,39,McKinley Dixon,For My Mama and Anyone Who Look Like Her,7 May 2021,"Jazz Rap, Southern Hip Hop, Conscious Hip Hop",3.64,1773.0,21.0
2479,959,39,40,Jaubi,Nafs at Peace,28 May 2021,"Spiritual Jazz, Jazz Fusion",3.65,1965.0,9.0


In [189]:
# processing to check similar album titles

from collections import defaultdict

SIM_THRES = 0.2
NOT_FOUND = 'NOT_FOUND'

def get_word_freq(l):
    d = defaultdict(int)
    for w in l:
        d[w] += 1
    return d

def freq_match(d1,d2):
    n = len(d1)
    sim = 0
    for k,v in d1.items():
        if k in d2 and d2[k] == v:
            sim += 1
    return sim/n


def are_same_albums(s1,s2):
    s1 = s1.lower()
    s2 = s2.lower()
    freq1 = get_word_freq(s1.split(' '))
    freq2 = get_word_freq(s2.split(' '))
    sim1 = freq_match(freq1, freq2)
    sim2 = freq_match(freq2, freq1)
    if max(sim1,sim2) >= SIM_THRES:
        return True
    return False

In [250]:
# retrieve spotify album id from a given name
def get_spotify_album(name): 
    search_res = sp.search(q=name, type='album')
    if len(search_res['albums']['items']) > 0 and are_same_albums(search_res['albums']['items'][0]['name'], name):
        return search_res['albums']['items'][0]['uri']
    return NOT_FOUND

In [191]:
# retrieve tracks of a valid album id
def get_album_tracks(album_id):
    tracks = []
    for track in sp.album_tracks(album_id=album_id)['items']:
        tracks.append(track['uri'])
    return tracks

In [192]:
# retrieve audio features
def get_audio_features(tracks):
    return sp.audio_features(tracks)

# retrieve audio analysis
def get_audio_analysis(track_id):
    return sp.audio_analysis(track_id)

In [220]:
# columns from different data sources
audio_features_cols = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature']
song_meta_cols = ['name', 'duration_ms', 'popularity', 'type']
rym_columns = ['Album', 'Date', 'Genres', 'RYM Rating', 'Ratings', 'Reviews']

In [255]:
# get audio features
track_features = []

def add_keys(dfrom, dto, keys):
    for key in keys:
        if key in dfrom:
            dto[key.lower()] = dfrom[key]
        else:
            dto[key.lower()] = None

switch = False
for index, row in rym_df.iterrows():
    # if row['Album'] == 'The Main Ingredient':
    #     print('dsfg')
    #     switch = True
    #     continue
    # if switch:
    #     print(row['Album'])
    #     album_id = get_spotify_album(row['Album'])
    #     print(album_id)
    album_id = get_spotify_album(row['Album'])
    if album_id != NOT_FOUND:
        tracks = get_album_tracks(album_id=album_id)
        curr_features = sp.audio_features(tracks)
        for d in curr_features:
            if d and set(audio_features_cols) == set(d.keys()):
                # add rym columns
                add_keys(row, d, rym_columns)
                # add song metadata
                song_meta = sp.track(d['id'])
                add_keys(song_meta, d, song_meta_cols)
                track_features.append(d)
                did,aid = d['id'], row['Album']
                print(f'Songs added for {did}, {aid}')

Giant Steps


KeyboardInterrupt: 

In [242]:
track_features

[]

In [212]:
album_id = get_spotify_album('The Dark Side of the Moon')
tracks = get_album_tracks(album_id=album_id)
tracks_feature = get_audio_features(tracks)
temp = sp.track(tracks[0])

In [236]:
len(track_features)

14363

In [239]:
df1 = pd.DataFrame(track_features)
df1.to_csv('../data/merge-data-1.csv', sep='~')

In [187]:
columns = []
tabular_features = defaultdict(list)
for track in track_features:
    if len(track) > 0 and track[0]:
        for k,v in track[0].items():
            tabular_features[k].append(v)
song_feature_df = pd.DataFrame(tabular_features)
song_feature_df.tail()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
1779,0.398,0.655,0,-10.301,1,0.0500,0.0261,0.000082,0.5670,0.1460,90.000,audio_features,7u3s0S0Xn4NNezPTtnKrkw,spotify:track:7u3s0S0Xn4NNezPTtnKrkw,https://api.spotify.com/v1/tracks/7u3s0S0Xn4NN...,https://api.spotify.com/v1/audio-analysis/7u3s...,438497,4
1780,0.376,0.868,1,-5.766,1,0.0878,0.1080,0.000112,0.1690,0.4090,99.777,audio_features,4Nv0jpNVsCMgDnhOlq8cUm,spotify:track:4Nv0jpNVsCMgDnhOlq8cUm,https://api.spotify.com/v1/tracks/4Nv0jpNVsCMg...,https://api.spotify.com/v1/audio-analysis/4Nv0...,194400,4
1781,0.664,0.477,0,-10.530,1,0.0551,0.3940,0.000041,0.1360,0.2730,100.046,audio_features,1ZP4Qxb7USL3EezbxfpOVF,spotify:track:1ZP4Qxb7USL3EezbxfpOVF,https://api.spotify.com/v1/tracks/1ZP4Qxb7USL3...,https://api.spotify.com/v1/audio-analysis/1ZP4...,223000,4
1782,0.414,0.800,6,-6.154,0,0.1230,0.1990,0.000000,0.3160,0.4270,128.647,audio_features,26NekpDz9TlmOzB2KtX3Kx,spotify:track:26NekpDz9TlmOzB2KtX3Kx,https://api.spotify.com/v1/tracks/26NekpDz9Tlm...,https://api.spotify.com/v1/audio-analysis/26Ne...,280955,4
1783,0.165,0.197,6,-16.400,1,0.0346,0.9420,0.498000,0.0895,0.0749,100.856,audio_features,3aD10lO7Runy5cpN9YaoNz,spotify:track:3aD10lO7Runy5cpN9YaoNz,https://api.spotify.com/v1/tracks/3aD10lO7Runy...,https://api.spotify.com/v1/audio-analysis/3aD1...,138670,4


In [198]:
audio_columns = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature']
rym_columns = ['Album', 'Date', 'Genres', 'RYM Rating', 'Ratings', 'Reviews']


In [204]:
print(f'Songs added for {columns}')

Songs added for ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature']


In [208]:
rym_df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Rank', 'Artist', 'Album', 'Date',
       'Genres', 'RYM Rating', 'Ratings', 'Reviews'],
      dtype='object')